In [1]:
import os

In [2]:
%pwd

'c:\\Users\\athar\\Projects\\Natural_Language_Processing\\Text-Summarizer-Project\\research'

In [3]:
os.chdir("C:/Users/athar/Projects/Natural_Language_Processing/Text-Summarizer-Project")

In [4]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path 
    data_path: Path 
    model_ckpt: Path 
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    per_device_eval_batch_size: int
    weight_decay: float
    logging_steps: int
    eval_strategy: str
    eval_steps: int
    save_steps: int
    gradient_accumulation_steps: int
    report_to: str
    fp16: bool
    gradient_checkpointing: bool

In [5]:
from TextSummarizer.constants import *
from TextSummarizer.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    """
    A class to manage configuration settings for a project.
    This class handles loading YAML configuration files, creating necessary directories,
    and providing configuration objects for specific components of the project.
    """
    
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,  # Default path for the main configuration file
        params_filepath=PARAMS_FILE_PATH  # Default path for the parameters file
    ):
        """
        Initialize the ConfigurationManager instance.

        Args:
            config_filepath (str): Path to the main configuration file (YAML).
            params_filepath (str): Path to the parameters file (YAML).
        """
        # Load the YAML configuration files into Python dictionaries
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        # Log the loaded configuration for debugging purposes
        print("Loaded Config:", self.config)
        
        # Create the directory specified in the `artifacts_root` key of the configuration
        create_directories([self.config.artifacts_root])
    
    def get_model_trainer_config(self)->ModelTrainerConfig:
        config=self.config.model_trainer
        params=self.params.TrainingArguments
        
        create_directories([config.root_dir])
        
        model_trainer_config= ModelTrainerConfig(
            root_dir= config.root_dir,
            data_path= config.data_path, 
            model_ckpt= config.model_ckpt, 
            num_train_epochs= params.num_train_epochs,
            warmup_steps= params.warmup_steps,
            per_device_train_batch_size= params.per_device_train_batch_size,
            per_device_eval_batch_size= params.per_device_eval_batch_size,
            weight_decay= params.weight_decay,
            logging_steps= params.logging_steps,
            eval_strategy= params.eval_strategy,
            eval_steps= params.eval_steps,
            save_steps= params.save_steps,
            gradient_accumulation_steps= params.gradient_accumulation_steps,
            report_to= params.report_to,
            fp16= params.fp16,
            gradient_checkpointing= params.gradient_checkpointing 
        )
        
        return model_trainer_config

In [7]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch 

c:\Users\athar\Projects\Natural_Language_Processing\Text-Summarizer-Project\testS\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-01-29 19:31:19,641: INFO: config: PyTorch version 2.5.1 available.]


In [8]:
class ModelTrainer:
    def __init__(self,config: ModelTrainerConfig):
        self.config= config
        
    def train(self):
        device='cuda' if torch.cuda.is_available() else 'cpu'
        tokenizer= AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus=AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator=DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)
        
        #load data
        dataset_samsum_pt=load_from_disk(self.config.data_path)
        
        trainer_args=TrainingArguments(
            output_dir= self.config.root_dir,
            num_train_epochs= self.config.num_train_epochs,
            warmup_steps= self.config.warmup_steps,
            per_device_train_batch_size= self.config.per_device_train_batch_size,
            per_device_eval_batch_size= self.config.per_device_eval_batch_size,
            weight_decay= self.config.weight_decay,
            logging_steps= self.config.logging_steps,
            eval_strategy= self.config.eval_strategy,
            eval_steps= self.config.eval_steps,
            save_steps= int(self.config.save_steps),
            gradient_accumulation_steps= self.config.gradient_accumulation_steps,
            report_to= self.config.report_to,
            fp16= self.config.fp16,
            gradient_checkpointing= self.config.gradient_checkpointing
        )
        
        trainer= Trainer(model=model_pegasus, args=trainer_args, tokenizer=tokenizer,
                 data_collator=seq2seq_data_collator, train_dataset=dataset_samsum_pt['train'],
                 eval_dataset=dataset_samsum_pt['validation'])
        
        trainer.train()
        
        #save the model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        # save the tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))

In [9]:
try:
    # Creating an object of the ConfigurationManager class  
    config = ConfigurationManager()  

    # Calling the get_model_trainer_config() method from the ConfigurationManager object  
    # This retrieves the model training configuration  
    model_trainer_config = config.get_model_trainer_config()  

    # Creating an object of the ModelTrainer class and passing the retrieved configuration as an argument  
    model_trainer_config = ModelTrainer(config=model_trainer_config)  

    # Calling the train() method from the ModelTrainer object to start the model training process  
    model_trainer_config.train()  
except Exception as e:
    raise e

[2025-01-29 19:31:19,970: INFO: common: yaml file:config\config.yaml  loaded successfully]
[2025-01-29 19:31:19,975: INFO: common: yaml file:params.yaml  loaded successfully]
Loaded Config: {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'data_validation': {'root_dir': 'artifacts/data_validation', 'STATUS_FILE': 'artifacts/data_validation/status.txt', 'ALL_REQUIRED_FILES': ['train', 'test', 'validation']}, 'data_transformation': {'root_dir': 'artifacts/data_transformation', 'data_path': 'artifacts/data_ingestion/samsum_dataset', 'tokenizer_name': 'google/pegasus-cnn_dailymail'}, 'model_trainer': {'root_dir': 'artifacts/model_trainer', 'data_path': 'artifacts/data_transformation/samsum_dataset', 'model_ckpt': 'google/pegasus-cnn_dailymail'}}
[2025-01-29 1

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\athar\AppData\Local\Temp\ipykernel_19056\605201895.py:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer= Trainer(model=model_pegasus, args=trainer_args, tokenizer=tokenizer,
  0%|          | 10/14732 [00:51<17:23:18,  4.25s/it]

{'loss': 3.9899, 'grad_norm': 20.56690788269043, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.0}


  0%|          | 20/14732 [01:28<14:23:12,  3.52s/it]

{'loss': 2.5735, 'grad_norm': 28.111074447631836, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.0}


...

{'loss': 1.3994, 'grad_norm': 31.65960121154785, 'learning_rate': 4.2158516020236085e-08, 'epoch': 1.0}


100%|█████████▉| 14730/14732 [70:08:44<00:10,  5.06s/it]

{'loss': 1.5692, 'grad_norm': 8.234221458435059, 'learning_rate': 7.026419336706015e-09, 'epoch': 1.0}


100%|██████████| 14732/14732 [70:08:55<00:00,  5.19s/it]c:\Users\athar\Projects\Natural_Language_Processing\Text-Summarizer-Project\testS\lib\site-packages\transformers\modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(
100%|██████████| 14732/14732 [70:09:28<00:00, 17.14s/it]


{'train_runtime': 252567.9708, 'train_samples_per_second': 0.058, 'train_steps_per_second': 0.058, 'train_loss': 1.5769180257521729, 'epoch': 1.0}
